<a href="https://colab.research.google.com/github/Abdul-AI-creator/Abdul-AI-creator/blob/main/Pytorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feed Forward Neural Network

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

**Device Configuration**

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

**Hyper Parameters**

In [10]:
input_size =784
hidden_size = 500
n_classes = 100
batch_size=100
n_epochs = 5
learning_rate = 0.001

**MNIST dataset**

In [5]:
train_dataset = torchvision.datasets.MNIST(root='../../data',download=True ,
                                           train=True ,
                                           transform=transforms.ToTensor())
train_dataset

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 39.5MB/s]


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.34MB/s]


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 9.38MB/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 1.93MB/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



Dataset MNIST
    Number of datapoints: 60000
    Root location: ../../data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [8]:
test_dataset = torchvision.datasets.MNIST(root = '../../data',
                                          train=False,transform=transforms.ToTensor())
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: ../../data
    Split: Test
    StandardTransform
Transform: ToTensor()

**Data Loaders**

In [11]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset ,batch_size=batch_size ,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset ,batch_size=batch_size,shuffle=False)
print(train_loader)
test_loader

**Fully Connected Neural Network with one Hidden Layer**

In [18]:
class NeuralNet(nn.Module):
  def __init__(self , input_size ,hidden_size ,n_classes):
    super(NeuralNet, self).__init__()                      # Super() class that inherit all the methods and properties from another class.
    self.fc1 = nn.Linear(input_size,hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size ,n_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out=self.fc2(out)
    return out

In [20]:
model = NeuralNet(input_size ,hidden_size ,n_classes).to(device)
model

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=100, bias=True)
)

**Loss and Optimizer**

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr =learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

# Train the model

In [24]:
total_step =len(train_loader)
for epoch in range(n_epochs):
  for i ,(images ,labels) in enumerate(train_loader):
    # move Tensor to configure Device
    images =images.reshape(-1 ,28*28).to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs ,labels)

    # Backward an optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 ==0:
      print("Epoch [{}/{}] ,Step [{}/{}],Loss: {:.4f}".format(epoch + 1,n_epochs ,i+1 ,total_step ,loss.item()))

Epoch [1/5] ,Step [100/600],Loss: 0.5673
Epoch [1/5] ,Step [200/600],Loss: 0.4247
Epoch [1/5] ,Step [300/600],Loss: 0.1121
Epoch [1/5] ,Step [400/600],Loss: 0.2022
Epoch [1/5] ,Step [500/600],Loss: 0.2137
Epoch [1/5] ,Step [600/600],Loss: 0.2521
Epoch [2/5] ,Step [100/600],Loss: 0.0610
Epoch [2/5] ,Step [200/600],Loss: 0.1942
Epoch [2/5] ,Step [300/600],Loss: 0.1455
Epoch [2/5] ,Step [400/600],Loss: 0.1352
Epoch [2/5] ,Step [500/600],Loss: 0.0551
Epoch [2/5] ,Step [600/600],Loss: 0.0756
Epoch [3/5] ,Step [100/600],Loss: 0.1229
Epoch [3/5] ,Step [200/600],Loss: 0.0654
Epoch [3/5] ,Step [300/600],Loss: 0.1562
Epoch [3/5] ,Step [400/600],Loss: 0.0734
Epoch [3/5] ,Step [500/600],Loss: 0.1175
Epoch [3/5] ,Step [600/600],Loss: 0.0713
Epoch [4/5] ,Step [100/600],Loss: 0.0301
Epoch [4/5] ,Step [200/600],Loss: 0.1490
Epoch [4/5] ,Step [300/600],Loss: 0.0254
Epoch [4/5] ,Step [400/600],Loss: 0.0296
Epoch [4/5] ,Step [500/600],Loss: 0.1206
Epoch [4/5] ,Step [600/600],Loss: 0.0670
Epoch [5/5] ,Ste

**test the model We don't need to compute gradient for memory efficient.**

In [27]:
with torch.no_grad():
  correct=0
  total =0
  for images ,labels in test_loader:
    images = images.reshape(-1 ,28*28).to(device)
    labels = labels.to(device)
    outputs =model(images)
    _ , predicted = torch.max(outputs.data , 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

torch.save(model.state_dict() ,'model.csv')

Accuracy of the network on the 10000 test images: 97.85 %
